In [23]:
from selenium import webdriver

from bs4 import BeautifulSoup as bs
import bs4
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from lxml import etree
import csv

### 스크래퍼 클래스 선언

In [2]:
class WebtoonParser(object):
    def __init__(self):
        self.driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
        ### 월,화,수,목,금,토,일,매일,완결
        self.weekend = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun','dailyPlus','finish']
        #self.weekend = ['mon']
        self.url = 'https://comic.naver.com/webtoon?tab='
        self.web = []
        self.image = []
        self.webtoon_url = {}
        self.webtoon_week = {}
    def excute(self):
        for category in self.weekend:
            nw_url = self.url + category
            driver = self.driver
            driver.get(nw_url)

            sleep(2)
            # 스크롤 높이 가져옴
            self.scrolling(driver)

            sleep(2)

            html = driver.page_source
            soup = bs(html, 'html.parser')

            ### 이미지 추출
            for title in soup.findAll('li', {'class':'item'}):
                self.recur_webtoon(title,category)
                img = title.findAll('img')
                # print("이미지 탐색 중")
                self.image.extend(img)
                # print("이미지 탐색 완료")
            

            webtoon_list = soup.find('ul', {'class':'ContentList__content_list--q5KXY'})
            self.total = len(webtoon_list)

    ### 타이틀 , 작가, 평점 추출
    def recur_webtoon(self,tag,week):
        for child in tag.descendants:
            if type(child) == bs4.element.NavigableString:
                if child == '청유물' or child == '신작' or child == '휴재' or child == '완결' or child == 'UP' or child == 'NEW' or child == '유료작품':
                    continue
                self.web.append(child)
            else:
                id_ = child.find('a')
                if id_ == None:
                    continue
                if id_['class'][0] == 'ContentTitle__title_area--x24vt':
                    id_text = id_.find('span', {'class':'ContentTitle__title--e3qXt'}).text
                    self.webtoon_url[id_text] = id_['href']
                    self.webtoon_week[id_text] = week

    def scrolling(self, driver):
        last_height = driver.execute_script("return document.body.scrollHeight")
        scroll_division = 10
        while True:
            
            print("스크롤 중")
            # 끝까지 스크롤 다운
            for i in range(1,scroll_division+1):
                driver.execute_script(f"window.scrollTo(0,(document.body.scrollHeight/{scroll_division})*{i})")
                sleep(2)
            sleep(2)
            # 스크롤 다운 후 스크롤 높이 다시 가져옴
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height



### 실행

In [3]:
abc = WebtoonParser()
abc.excute()

스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중


In [5]:
print(len(abc.web))
print(len(abc.image))

10512
2518


### title, author, rating dictionary 저장

In [6]:
lt1 = abc.web

webtoon = []
for i in range(0,len(lt1),4):
    dic = {}
    try:    
        dic['title'] = lt1[i]
        dic['author'] = lt1[i+1]
        dic['rating'] = lt1[i+3]
    except:
        print("abc")

    webtoon.append(dic)

### title image 추가

In [7]:
for a, i in enumerate(abc.image):
    for b, j in enumerate(webtoon):
        if i['alt'] == j['title']:
            webtoon[b]['image'] = i['src']
            webtoon[b]['week'] = abc.webtoon_week[j['title']]
            

### url 추가

In [8]:
for a, i in enumerate(abc.webtoon_url):
    for b, j in enumerate(webtoon):
        if i == j['title']:
            webtoon[b]['url'] = abc.webtoon_url[i]

### 제목 [드라마원작] 제거

In [9]:
for a,i in enumerate(webtoon):
    try:
        webtoon[a]['title'] = webtoon[a]['title'].replace("[드라마원작] ", "")
    except:
        webtoon[a]['url'] = "none"

### csv 파일로 저장

In [10]:
with open('./data/webtoon_1.csv', 'w', encoding='utf-8', newline='') as f:
    fieldnames = ['title', 'author', 'rating', 'image', 'url', 'week']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for i in webtoon:
        writer.writerow(i)
        


### 장르 크롤링

In [33]:

webtoon_list = []

with open('./data/webtoon_1.csv', 'r', encoding='utf-8', newline='') as f:
    fieldnames = ['title', 'author', 'rating', 'image','url','week']
    reader = csv.DictReader(f, fieldnames=fieldnames)
    
    for n, row in enumerate(reader):
        if n == 0:
            continue

        tmp = {}
        tmp['title'] = row['title']
        tmp['author'] = row['author']
        tmp['rating'] = row['rating']
        tmp['image'] = row['image']
        url = 'https://comic.naver.com'+ row['url']
        tmp['url'] = url

        driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
        driver.get(url)
        sleep(1)

        html = driver.page_source
        soup = bs(html, 'html.parser')
        
        try:
            a = soup.find('div', {'class':'TagGroup__tag_group--uUJza'}).text
        except:
            tmp['genre'] = '청소년이용불가'
            webtoon_list.append(tmp)
            driver.close()
            continue

        tmp['genre'] = a
        tmp['week'] = row['week']
        
        webtoon_list.append(tmp)

        driver.close()


### 최종 형태 저장

In [ ]:
with open('./data/webtoon_com_1.csv', 'w', encoding='utf-8', newline='') as f1:
    fieldnames1 = ['title', 'author', 'rating', 'image', 'url','genre','week']
    writer1 = csv.DictWriter(f1, fieldnames=fieldnames1)
    writer1.writeheader()
    for i in webtoon_list:
        writer1.writerow(i)

In [24]:
tmp_list = []
with open('./data/webtoon_com.csv', 'r', encoding='utf-8', newline='') as f:
    fieldnames = ['title', 'author', 'rating', 'image','url','genre']
    reader = csv.DictReader(f, fieldnames=fieldnames)
    for n , row in enumerate(reader):
        if n == 0:
            continue
        
        tmp_list.append(row)

In [49]:

result_list = []

driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))

for n, webtoon_url in enumerate(tmp_list):
    tmp = {}
    
    url = webtoon_url['url']
    
    driver.get(url)
    sleep(0.1)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    try:
        week = soup.find('em', {'class':'ContentMetaInfo__info_item--utGrf'}).text
    except:
        week = "기타"
    
    print(f'{(n/len(tmp_list)) * 100:.3f} / {webtoon_url["title"]} : {week}')
    tmp['title'] = webtoon_url['title']
    tmp['author'] = webtoon_url['author']   
    tmp['rating'] = webtoon_url['rating']
    tmp['image'] = webtoon_url['image']
    tmp['url'] = webtoon_url['url']
    tmp['genre'] = webtoon_url['genre']
    tmp['week'] = week
    
    result_list.append(tmp)
    # driver.close()
    

0.000 / 참교육 : 월요웹툰 ∙ 15세 이용가
0.039 / 뷰티풀 군바리 : 월요웹툰 ∙ 15세 이용가
0.077 / 신의 탑 : 월요웹툰 ∙ 12세 이용가
0.116 / 똑 닮은 딸 : 월요웹툰 ∙ 15세 이용가
0.154 / 백수세끼 : 월요웹툰 ∙ 전체연령가
0.193 / 퀘스트지상주의 : 월요웹툰 ∙ 15세 이용가
0.232 / 장씨세가 호위무사 : 월요웹툰 ∙ 12세 이용가
0.270 / 윈드브레이커 : 기타
0.309 / 소녀의 세계 : 월요웹툰 ∙ 전체연령가
0.347 / 버림받은 왕녀의 은밀한 침실 : 기타
0.386 / 신화급 귀속 아이템을 손에 넣었다 : 월요웹툰 ∙ 15세 이용가
0.425 / 우아한 욕망 : 기타
0.463 / 별난식당 : 월요웹툰 ∙ 전체연령가
0.502 / 절대검감 : 월요웹툰 ∙ 12세 이용가
0.541 / 앵무살수 : 월요웹툰 ∙ 15세 이용가
0.579 / 퍼니게임 : 월요웹툰 ∙ 15세 이용가
0.618 / ㅋㅋ단편.zip : 월, 목 연재 ∙ 전체연령가
0.656 / 만렙돌파 : 월요웹툰 ∙ 15세 이용가
0.695 / 잔불의 기사 : 월요웹툰 ∙ 15세 이용가
0.734 / 나 없는 단톡방 : 월요웹툰 ∙ 12세 이용가
0.772 / 순정말고 순종 : 월요웹툰 ∙ 15세 이용가
0.811 / 물어보는 사이 : 월요웹툰 ∙ 15세 이용가
0.849 / 개같이 탈출 : 월요웹툰 ∙ 전체연령가
0.888 / 리턴 투 플레이어 : 월요웹툰 ∙ 15세 이용가
0.927 / 좀비묵시록 82-08 : 월요웹툰 ∙ 15세 이용가
0.965 / 히어로메이커 : 월요웹툰 ∙ 전체연령가
1.004 / 천재 타자가 강속구를 숨김 : 월요웹툰 ∙ 전체연령가
1.042 / 신군 : 월요웹툰 ∙ 15세 이용가
1.081 / 파견체 : 월요웹툰 ∙ 12세 이용가
1.120 / 루루라라 우리네 인생 : 기타
1.158 / 칼가는 소녀 : 월요웹툰 ∙ 12세 이용가
1.197 / 시한부의 아이까지 뺏으려 합니다 : 월요웹툰 ∙ 전체연

In [72]:
weekend = ['월','화','수','목','금','토','일','매일','완결']
for i in result_list[0:3]:
    a = i['week'].split('∙')[0]
    i['week'] = ""
    
    for j in a:
        if j in weekend:
            
    
    print(i)
            
    

{'title': '참교육', 'author': '채용택 / 한가람', 'rating': '9.87', 'image': 'https://image-comic.pstatic.net/webtoon/758037/thumbnail/thumbnail_IMAG21_15cb2611-34c0-4f02-a689-41d0b1016579.jpg', 'url': 'https://comic.naver.com/webtoon/list?titleId=758037', 'genre': '#액션#블루스트링#사이다#학원물#다크히어로#먼치킨', 'week': '월'}
{'title': '뷰티풀 군바리', 'author': '설이 / 윤성원', 'rating': '9.81', 'image': 'https://image-comic.pstatic.net/webtoon/648419/thumbnail/thumbnail_IMAG21_d9398229-cbfd-47dc-9208-0a6fb936f3a7.jpg', 'url': 'https://comic.naver.com/webtoon/list?titleId=648419', 'genre': '#드라마#직업드라마#밀리터리', 'week': '월'}
{'title': '신의 탑', 'author': 'SIU', 'rating': '9.88', 'image': 'https://image-comic.pstatic.net/webtoon/183559/thumbnail/thumbnail_IMAG21_5f3fec31-5c95-4afe-a73f-3046288edb47.jpg', 'url': 'https://comic.naver.com/webtoon/list?titleId=183559', 'genre': '#판타지#세계관#이능력배틀물#소년왕도물#먼치킨', 'week': '월'}


In [76]:
test = '월 목 연재'


for i in test:
    tmp = []
    if i in weekend:
        print(i)
        tmp.join(i)



월


AttributeError: 'list' object has no attribute 'join'